In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/Tesi"

/content/drive/MyDrive/Tesi


In [ ]:
%pip install mne --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 4.3 MB/s 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mne
import pickle
import os 
from sklearn.utils import shuffle
import cv2
import scipy.signal as signal
from sklearn.model_selection import train_test_split
import h5py

## Salvataggio di ogni seizure per il k fold cross val

In [ ]:
train_test = [4,3,4,3,2,2,3,2,5,5,2,1,3,2,2,2,4,3,5,4,4,3,4,4,7,6,3,2,2,1,3,2,3,2,2,2,2,1,3,3]
X_seiz_train = None
X_seiz_test = None
y_seiz_train = []
y_seiz_test = []


cardSeiz = 0 # cardinalità di seizure per ogni paziente
count = 0
index = 0
train_flag = True
patient = "chb01"
with open("chb-mit-annotation-with-seizure.txt") as f:
    lines = f.readlines()
    for line in lines:
      instance = []
      print(line)
      el = line.split(" ")
      if (el[0].split("/")[0]) != patient:
        print("nuovo paziente")
        count = 0
        patient = el[0].split("/")[0]


      cardSeiz = (len(el)-1)/2

      start_onset = int(el[1])

      file = "chb-mit-scalp-eeg-database-1.0.0/" + str(el[0])
      data = mne.io.read_raw_edf(file)
      channels = data.ch_names

      # Resampling data
      data.resample(sfreq=256)

      # Se devo prendere una sola seizure ed è anche la prima
      if cardSeiz == 1 and int(el[2]) == 1:
        start_seiz = 0 #Continua, croppa due avolte e taglia due volte
        if (start_onset-start_seiz) % 6 != 0: # La lunghezza delle istanze deve essere multipla di 6 poichè alla rete neurale andranno segmenti di lunghezza 6
          start_seiz = (start_onset-start_seiz) % 6
        instance.append(data.crop(start_seiz, start_onset))
        
      # Se devo prendere una sola seizure nella registrazione ma questa non è la prima che incontro devo tagliare 1300 secondi prima dell'onset (ho messo 1302 perchè divisibile perfettamente per 6) 
      elif cardSeiz == 1 and int(el[2]) != 1:
        start_seiz = start_onset - 1302 
        instance.append(data.crop(start_seiz, start_onset))
      # Se devo prendere due seizure o tre 
      elif cardSeiz >= 2:
        data2 = data.copy() 
        data3 = data.copy() 
        # Se la prima che devo prendere è proprio la prima allora prendo la registrazione dall'inizio, altrimenti da 1300 secondi prima dell'onset (ho messo 1302 perchè divisibile perfettamente per 6)
        if int(el[3]) == 1 or int(el[4]) == 1: 
          start_seiz = 0
          if (start_onset-start_seiz) % 6 != 0: # La lunghezza delle istanze deve essere multipla di 6 poichè alla rete neurale andranno segmenti di lunghezza 6
            start_seiz = (start_onset-start_seiz) % 6
          instance.append(data.crop(start_seiz, start_onset))
        else: 
          start_seiz = start_onset - 1302 
          instance.append(data.crop(start_seiz, start_onset))

        start_onset2 = int(el[2])
        start_seiz2 = start_onset2 - 1302
        instance.append(data2.crop(start_seiz2, start_onset2))
        
        if cardSeiz == 3:
          start_onset3 = int(el[3])
          start_seiz3 = start_onset3 - 1302
          instance.append(data3.crop(start_seiz3, start_onset3))
        del data2, data3
      
        
      for data in instance:

        # Filter
        data.load_data()
        filt_raw = data.filter(l_freq=8., h_freq=13, picks='eeg', method = 'fir', verbose = False);

        # Differentiating
        diff = abs(np.diff(filt_raw.get_data(picks=['eeg'])));

        # Creazione e divisione degli spettrogrammi in blocchi da un secondo ciascuno
        block = None
        j=0
        
        # la fase preictale inizia o a 300 o a 900 secondi prima dell'attacco
        preictal_300 = ((len(diff[0])//256) - 300)        
        # j cicla su ogni secondo del segnale
        while j < (len(diff[0])//256)-5 :
          # i cicla su ogni canale
          for i in range(0, len(channels[:22])):
              start = (j*256) 
              stop = start+(256*6)
              
              _, _, Pxx =signal.spectrogram(diff[i,start:stop], nfft=256, fs=256, noverlap=0)
              spect = cv2.flip(np.uint8(10*np.log10(Pxx)),0)
              if(block is None):
                block = np.array([spect[-40:]])
              else:
                block = np.append(block, np.array([spect[-40:]]), axis=0)
          
          # Riempimento di train e test diviso come da paper
          if (X_seiz_train is None):
            X_seiz_train = np.array([block])
          elif (X_seiz_train is not None):
            X_seiz_train = np.append(X_seiz_train, np.array([block]), axis=0)
    
          # Se ci troviamo nella fase preictale 300
          if j >= preictal_300:
            y_seiz_train.append(1)
            
          else: # se ci troviamo nella fase interictale
            y_seiz_train.append(0)
        

          if (stop//256) == preictal_300 -1:
            print("Stop = ", str(stop//256))          
            j = j + 6
          else:
            j = j + 1

          block = None
          
        count = count + 1
        
        y_seiz_train = np.asarray(y_seiz_train)

        X_seiz_train = X_seiz_train.reshape((len(X_seiz_train), 6, 40, 22))
        with open("./chb_seiz_for_fold/" + str(patient) + '_X_y_seizure' + str(count) + '.pickle', "wb") as file:
          pickle.dump((X_seiz_train, y_seiz_train), file)
          file.close()
        print(f"{patient}, seizure n° {count} , X_shape = {X_seiz_train.shape}, y_shape = {y_seiz_train.shape}")
        print("quanti 1 ci sono nel train? ", np.count_nonzero(y_seiz_train)) 
        print(f"{y_seiz_train[0]}, {y_seiz_train[-1]},{y_seiz_train[-294]}, {y_seiz_train[-295]}, {y_seiz_train[-296]}")
        X_seiz_train = None
        y_seiz_train = []

##Creazione dei fold

In [ ]:
import h5py
patients = ["01", "03", "04", "05", "06", "07", "08", "09", "10", "12", "13", "14", "15", "16", "17", "18", "20", "21", "22", "23"]
splitting = [3,1,3, 3,1,3, 1,1,2 ,2,1,2 ,3,2,5, 1,1,1, 2,1,2 ,1,1,2 ,3,1,3 ,3,2,4, 3,1,3, 3,1,4, 4,3,6, 2,1,2 ,1,1,1, 2,1,2, 2,1,2, 1,1,2, 1,1,1, 2,1,3]
for p in patients:
  patient = "chb" + str(p)
  count_seiz = 0
  n_fold = 0
  X_tot = []
  y_tot = []

  for file in (os.listdir("./chb_seiz_for_fold/")):
   
    if (str(file).split("_")[0] == patient):
      count_seiz = count_seiz + 1
      with open("./chb_seiz_for_fold/" + str(file), 'rb') as file:
        X_train, y_train = pickle.load(file)
        X_tot.append(X_train)
        y_tot.append(y_train)

  if count_seiz >= 4:
    n_fold = 5
  else:
    n_fold = 3
  print("Numero fold = " + str(n_fold))
  for k in range(0,n_fold):
    copy_X_tot = X_tot.copy()
    copy_y_tot = y_tot.copy()
    X_test = None
    X_train = None
    X_val = None
    y_train = None
    y_test = None
    y_val = None

    # riempio il test
    for i in range(0,splitting[2]):
      if k >= len(copy_X_tot):
        if X_test is None:
          X_test = np.array(copy_X_tot[i])
          y_test = np.array(copy_y_tot[i])
        else:
          X_test = np.append(X_test, np.array(copy_X_tot[i]), axis = 0)
          y_test = np.append(y_test,np.array(copy_y_tot[i]), axis = 0) 
        del  copy_X_tot[i], copy_y_tot[i]  
      else:
        if X_test is None:
          X_test = np.array(copy_X_tot[k])
          y_test = np.array(copy_y_tot[k])
        else:
          X_test = np.append(X_test, np.array(copy_X_tot[k]), axis = 0)
          y_test = np.append(y_test,np.array(copy_y_tot[k]), axis = 0)
        del copy_X_tot[k], copy_y_tot[k]

    # riempio il train
    for i in range(0,splitting[0]):
      if X_train is None:
        X_train = np.array(copy_X_tot[i])
        y_train = np.array(copy_y_tot[i])
      else:
        X_train = np.append(X_train, np.array(copy_X_tot[i]), axis = 0)
        y_train = np.append(y_train, np.array(copy_y_tot[i]), axis = 0)
    #riempio il val
    for i in range(splitting[0], splitting[1]+splitting[0]):
      if X_val is None:
        X_val = np.array(copy_X_tot[i])
        y_val = np.array(copy_y_tot[i])
      else:
        X_val = np.append(X_val, np.array(copy_X_tot[i]), axis = 0)
        y_val = np.append(y_val, np.array(copy_y_tot[i]), axis = 0)

    # Aggiungi lo shuffle al train 
    
    with open("./chb_seiz_for_fold/" + str(patient) + "_X_test_y_test_" + str(k) + "_fold.pickle", "wb") as file:
      pickle.dump((X_test, y_test), file)
      print(f" X test shape = {(X_test.shape)}")
      file.close()
      
    with h5py.File("./chb_seiz_for_fold/" + str(patient) + "_X_train_y_train_" + str(k) + "_fold.h5", 'w') as f:
      f.create_dataset(str(patient) + '_X_train', data=X_train, maxshape=(None, 6, 40, 22),chunks=True)
      f.create_dataset(str(patient) + '_y_train', data=y_train,  maxshape=(None,),chunks=True)
      print(f" X train shape = {(X_train.shape)} ")
      file.close()

    with open("./chb_seiz_for_fold/" + str(patient) + "_X_val_y_val_" + str(k) + "_fold.pickle", 'wb') as file:
      pickle.dump((X_val,y_val), file)
      print(f" X val shape = {X_val.shape} ")
      file.close()


    X_test = None
    X_train = None
    X_val = None
    y_train = None
    y_test = None
    y_val = None
    print("Scritto")
  del splitting[0:3]

  

## Dataset specific

In [ ]:
import h5py
from pathlib import Path
patients = ["01", "03", "04", "05", "06", "07", "08", "09", "10", "12", "13", "14", "15", "16", "17", "18", "20", "21", "22", "23"] #dividione train test 60% 40%, divisione train val 70% 30%- da chb01 a chb09 train, da chb10 a 14 val e da 15 a 23 test
print("Riempio il train")
X_train_tot = None
y_train_tot = None
X_val_tot = None
y_val_tot = None
X_test_tot = None
y_test_tot = None
for p in patients[:8]: #riempio il train
  patient = "chb" + str(p)



  for i in range(1,11): # 10 è il max numero di seizure che ho
    if Path("./chb_seiz_for_fold/" + str(patient) + "_X_y_seizure" +str(i) + ".pickle").is_file():
      with open("./chb_seiz_for_fold/" + str(patient) + "_X_y_seizure" + str(i) + ".pickle", 'rb') as file:
            X_train, y_train = pickle.load(file)

      print(f"Aperto il file: {patient}_X_y_seizure{i}.pickle")
      if (X_train_tot is None):
            X_train_tot = np.array(X_train)
            y_train_tot = np.array(y_train)
      elif (X_train_tot is not None):
            X_train_tot = np.append(X_train_tot, np.array(X_train), axis=0)
            y_train_tot = np.append(y_train_tot, np.array(y_train), axis=0)


with h5py.File("./chb_seiz_for_fold/datasetSpecific/X_train_y_train.h5", 'w') as f:
  f.create_dataset('X_train', data=X_train_tot, maxshape=(None, 6, 40, 22),chunks=True)
  f.create_dataset('y_train', data=y_train_tot,  maxshape=(None,),chunks=True)
  print(f" X train shape = {X_train_tot.shape} ")
  
  file.close()
del X_train_tot, y_train_tot

print("Riempio il val")
for p in patients[8:12]: #riempio il train
  patient = "chb" + str(p)



  for i in range(1,11):
    if Path("./chb_seiz_for_fold/" + str(patient) + "_X_y_seizure" + str(i) + ".pickle").is_file():
      with open("./chb_seiz_for_fold/" + str(patient) + "_X_y_seizure" + str(i) + ".pickle", 'rb') as file:
            X_train, y_train = pickle.load(file)

      print(f"Aperto il file: {patient}_X_y_seizure{i}.pickle")
      if (X_val_tot is None):
            X_val_tot = np.array(X_train)
            y_val_tot = np.array(y_train)
      elif (X_val_tot is not None):
            X_val_tot = np.append(X_val_tot, np.array(X_train), axis=0)
            y_val_tot = np.append(y_val_tot, np.array(y_train), axis=0)

with open("./chb_seiz_for_fold/datasetSpecific/X_val_y_val.pickle", 'wb') as file:
  pickle.dump((X_val_tot,y_val_tot), file)
  print(f" X val shape = {X_val_tot.shape} ")
  file.close()
del X_val_tot, y_val_tot

print("Riempio il test")
for p in patients[12:]: #riempio il train
  patient = "chb" + str(p)



  for i in range(1,11):
    if Path("./chb_seiz_for_fold/" + str(patient) + "_X_y_seizure" + str(i) + ".pickle").is_file():
      with open("./chb_seiz_for_fold/" + str(patient) + "_X_y_seizure" + str(i) + ".pickle", 'rb') as file:
            X_train, y_train = pickle.load(file)

      print(f"Aperto il file: {patient}_X_y_seizure{i}.pickle")
      if (X_test_tot is None):
            X_test_tot = np.array(X_train)
            y_test_tot = np.array(y_train)
      elif (X_test_tot is not None):
            X_test_tot = np.append(X_test_tot, np.array(X_train), axis=0)
            y_test_tot = np.append(y_test_tot, np.array(y_train), axis=0)

with open("./chb_seiz_for_fold/datasetSpecific/X_test_y_test.pickle", "wb") as file:
  pickle.dump((X_test_tot, y_test_tot), file)
  file.close()
  print(f" X test shape = {X_test_tot.shape} ")
    




##General approach

In [ ]:
import h5py
from pathlib import Path
patients = ["01", "03", "04", "05", "06", "07", "08", "09", "10", "12", "13", "14", "15", "16", "17", "18", "20", "21", "22", "23"] #dividione train test 60% 40%, divisione train val 70% 30%- da chb01 a chb09 train, da chb10 a 14 val e da 15 a 23 test
print("Riempio il train")
X_train_tot = None
y_train_tot = None
X_val_tot = None
y_val_tot = None
X_test_tot = None
y_test_tot = None
for p in patients: #riempio il train
  patient = "chb" + str(p)



  for i in range(1,14): # 10 è il max numero di seizure che ho
    if Path("./chb_seiz_for_fold/" + str(patient) + "_X_y_seizure" +str(i) + ".pickle").is_file():
      with open("./chb_seiz_for_fold/" + str(patient) + "_X_y_seizure" + str(i) + ".pickle", 'rb') as file:
            X_train, y_train = pickle.load(file)

      print(f"Aperto il file: {patient}_X_y_seizure{i}.pickle")
      if (X_train_tot is None):
            X_train_tot = np.array(X_train)
            y_train_tot = np.array(y_train)
      elif (X_train_tot is not None):
            X_train_tot = np.append(X_train_tot, np.array(X_train), axis=0)
            y_train_tot = np.append(y_train_tot, np.array(y_train), axis=0)


with h5py.File("./chb_seiz_for_fold/general/X_train_y_train.h5", 'w') as f:
  f.create_dataset('X_train', data=X_train_tot, maxshape=(None, 6, 40, 22),chunks=True)
  f.create_dataset('y_train', data=y_train_tot,  maxshape=(None,),chunks=True)
  print(f" X train shape = {X_train_tot.shape} ")
  
  file.close()
del X_train_tot, y_train_tot



